In [ ]:
import imageio
import os

def extract_and_save_frames_from_gif(gif_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Use imageio's get_reader to read the GIF file frame by frame
    with imageio.get_reader(gif_path) as reader:
        # Loop through each frame in the gif
        for i, frame in enumerate(reader):
            # Construct the filename for each frame
            frame_filename = f"frame_{i}.png"
            frame_path = os.path.join(output_folder, frame_filename)

            # Save each frame
            imageio.imwrite(frame_path, frame)

            print(f"Frame {i} saved as {frame_filename}")

# Usage
gif_path = 'Output/Output_3_Dataset_2/TEM_Data_img.gif'  
output_folder = 'Output/Output_3_Dataset_2/frames'  # Replace with your desired output path

extract_and_save_frames_from_gif(gif_path, output_folder)


In [ ]:
import cv2
import numpy as np
from skimage.restoration import denoise_wavelet, denoise_nl_means, estimate_sigma
from skimage.filters import median
from tqdm import tqdm
import os

def denoise_images(input_folder, output_base_folder):
   
    if not os.path.exists(output_base_folder):
        os.makedirs(output_base_folder)

    
    files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

    
    denoising_methods = {
        'wavelet': denoise_wavelet,
        'gaussian_blur': lambda img: cv2.GaussianBlur(img, (5, 5), 0),
        'median_filter': lambda img: median(img),
        'bilateral_filter': lambda img: cv2.bilateralFilter(img, 9, 75, 75),
        'non_local_means': lambda img: denoise_nl_means(img, h=1.15 * np.std(img))
    }

   
    for file in tqdm(files):
        # Read the image
        img_path = os.path.join(input_folder, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        for method_name, denoising_function in denoising_methods.items():
          
            denoised_img = denoising_function(img)

            # Create a subfolder for the method if it doesn't exist
            output_folder = os.path.join(output_base_folder, method_name)
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            # Save the denoised image
            denoised_img_path = os.path.join(output_folder, file)
            cv2.imwrite(denoised_img_path, (denoised_img * 255).astype('uint8'))

            print(f"Denoised image using {method_name} saved to {denoised_img_path}")


input_folder = 'Output/Output_3_Dataset_2/frames' 
output_base_folder = 'Output/Output_3_Dataset_2/Denoised_frames'  

denoise_images(input_folder, output_base_folder)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

# Load a pre-trained model from TensorFlow Hub
model = hub.load('https://tfhub.dev/captain-pool/esrgan-tf2/1')

def enhance_image(image_path, save_path):
    pil_image = Image.open(image_path).convert('L')
    original_size = pil_image.size  
    image = np.array(pil_image)
    rgb_image = np.stack((image,) * 3, axis=-1)
    
    # Convert to TensorFlow tensor and preprocess
    rgb_image = tf.cast(rgb_image, tf.float32) / 255.0
    rgb_image = tf.expand_dims(rgb_image, 0)

    # Apply the model
    enhanced_image = model(rgb_image)
    enhanced_image = tf.squeeze(enhanced_image, 0)

    # Resize to original dimensions
    resized_enhanced_image = tf.image.resize(enhanced_image, original_size, method=tf.image.ResizeMethod.BICUBIC)

    # Convert the output back to grayscale
    grayscale_output = tf.image.rgb_to_grayscale(resized_enhanced_image)

    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    save_image = tf.keras.preprocessing.image.array_to_img(grayscale_output)
    save_image.save(save_path)

    return grayscale_output

image_path = 'Datasets/RGB/Dataset_2/2725_OV_-10_1.jpg'  
save_path = 'Output/Output_3_Dataset_2/ESRGAN'  
enhanced_image = enhance_image(image_path, save_path)

# Display the result
plt.figure(figsize=(6, 6))
plt.imshow(enhanced_image.numpy().squeeze(), cmap='gray')  
plt.axis('off')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import os

def calculate_psnr(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_ssim(image1, image2):
    ssim_value, _ = ssim(image1, image2, full=True)
    return ssim_value

def load_image(image_path):
    return cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

def show_images_with_metrics(image_path1, folder_path, save_dir):
    img1 = load_image(image_path1)

    max_psnr_value = -1
    best_img_path = None
    best_img = None

    for filename in os.listdir(folder_path):
        image_path2 = os.path.join(folder_path, filename)
        if os.path.isfile(image_path2):
            img2 = load_image(image_path2)
            psnr_value = calculate_psnr(img1, img2)

            if psnr_value > max_psnr_value:
                max_psnr_value = psnr_value
                best_img_path = image_path2
                best_img = img2

    if best_img is not None:
        ssim_value = calculate_ssim(img1, best_img)

        plt.figure(figsize=(10, 5))

        # Display first image
        plt.subplot(1, 2, 1)
        plt.imshow(img1, cmap='gray')
        plt.title('Input TEM Image')
        plt.axis('off')

        # Display best matched image
        plt.subplot(1, 2, 2)
        plt.imshow(best_img, cmap='gray')
        plt.title('Our Result')
        plt.axis('off')

        plt.suptitle(f"PSNR Value: {max_psnr_value:.2f} dB, SSIM Value: {ssim_value:.2f}")

        # Save the figure
        save_path = os.path.join(save_dir, "Dataset_2_wavelet.jpg")
        plt.savefig(save_path)
        print(f"Image saved to {save_path}")

# Make sure the directory exists where you want to save the images
save_directory = 'Comparism Output'
os.makedirs(save_directory, exist_ok=True)

image_path1 = 'Datasets/JPG/Dataset_2/2725_OV_-10_1.jpg'
folder_path = 'Output/Output_3_Dataset_2/Denoised_frames/wavelet'

show_images_with_metrics(image_path1, folder_path, save_directory)
